In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Now, you can use the path to your Google Drive like this
prefix = '/content/drive/MyDrive/DS-MQTT/'


Mounted at /content/drive


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!git clone https://github.com/AbertayMachineLearningGroup/MQTT_ML.git

Cloning into 'MQTT_ML'...
remote: Enumerating objects: 29, done.
remote: Counting objects: 100% (29/29), done.
remote: Compressing objects: 100% (24/24), done.
remote: Total 29 (delta 7), reused 9 (delta 2), pack-reused 0
Receiving objects: 100% (29/29), 16.29 KiB | 5.43 MiB/s, done.
Resolving deltas: 100% (7/7), done.


In [ ]:
%cd MQTT_ML/notebooks

[Errno 2] No such file or directory: 'MQTT_ML/notebooks'
/content


In [ ]:
%cd MQTT_ML/

[Errno 2] No such file or directory: 'MQTT_ML/'
/content


In [ ]:
ls

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Now, you can use the path to your Google Drive like this
prefix = '/content/drive/MyDrive/DS-MQTT/'


In [ ]:
import pandas as pd
import numpy as np
import os
import argparse

In [ ]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC, LinearSVC
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import classification_report
from sklearn.neural_network import MLPClassifier
from xgboost import XGBClassifier

In [ ]:
# helper function: to convert string data to boolean

def str2bool(v):
  if v.lower() in ('yes', 'true', 't', 'y', '1'):
    return True
  elif v.lower() in ('no', 'false', 'f', 'n', '0'):
    return False
  else:
    raise argparse.ArgumentTypeError('Booloean value expected.')

In [ ]:
# data preprocessing

one_hot_encoder = None

def load_file(path, mode, is_attack = 1, label = 1, folder_name='Bi/', sliceno = 0, verbose = True):
    #global label_encoder
    global one_hot_encoder

    #attacker_ips = ['192.168.2.5']

    columns_to_drop_packet = ['timestamp', 'src_ip', 'dst_ip']
    columns_to_drop_uni = ['proto', 'ip_src', 'ip_dst']
    columns_to_drop_bi = ['proto', 'ip_src', 'ip_dst']

    if os.path.getsize(path)//10 ** 9 > 0:
        x = np.zeros((0,0))
        for chunk in pd.read_csv(path, chunksize=10 ** 6):
            chunk.drop(columns = columns_to_drop_packet, inplace = True)
            chunk = chunk[chunk.columns.drop(list(chunk.filter(regex='mqtt')))]

            chunk = chunk.fillna(-1)

            with open(folder_name + 'instances_count.csv','a') as f:
                f.write('{}, {} \n'.format(path, chunk.shape[0]))

            x_temp = chunk.loc[chunk['is_attack'] == is_attack]
            x_temp.drop('is_attack', axis = 1, inplace = True)
            #x_temp['protocol'] = label_encoder.transform(x_temp['protocol'])
            if one_hot_encoder == None:
                one_hot_encoder = OneHotEncoder(categorical_features=[0], n_values=30)
                x_temp = one_hot_encoder.fit_transform(x_temp).toarray()
            else:
                x_temp = one_hot_encoder.transform(x_temp).toarray()

            x_temp = np.unique(x_temp, axis = 0)

            if x.size == 0:
                x = x_temp
            else:
                x = np.concatenate((x, x_temp), axis = 0)
                x = np.unique(x, axis = 0)
    else:
        dataset = pd.read_csv(path)

        if mode == 1 or mode == 2:
            dataset = dataset.loc[dataset['is_attack'] == is_attack]
#            if is_attack == 0:
#                dataset = dataset.loc[operator.and_(dataset['ip_src'].isin(attacker_ips) == False, dataset['ip_dst'].isin(attacker_ips) == False)]
#            else:
#                dataset = dataset.loc[operator.or_(dataset['ip_src'].isin(attacker_ips), dataset['ip_dst'].isin(attacker_ips))]
#
        if mode == 0:
            dataset.drop(columns=[columns_to_drop_packet], inplace = True)
            dataset = dataset[dataset.columns.drop(list(dataset.filter(regex='mqtt')))]
        elif mode == 1:
            dataset.drop(columns = columns_to_drop_uni, inplace = True)
        elif mode == 2:
            dataset.drop(columns = columns_to_drop_bi, inplace = True)

        if verbose:
            print(dataset.columns)

        dataset = dataset.fillna(-1)

        if mode == 0:
            x = dataset.loc[dataset['is_attack'] == is_attack]
            x.drop('is_attack', axis=1, inplace=True)
            #x['protocol'] = label_encoder.transform(x['protocol'])
            if one_hot_encoder == None:
                one_hot_encoder = OneHotEncoder(categorical_features=[0], n_values=30)
                x = one_hot_encoder.fit_transform(x).toarray()
            else:
                x = one_hot_encoder.transform(x).toarray()
        else:
            x = dataset.values

    with open(folder_name + 'instances_count.csv','a') as f:
        f.write('all, {}, {} \n'.format(path, x.shape[0]))

    x = np.unique(x, axis = 0)

    with open(folder_name + 'instances_count.csv','a') as f:
        f.write('unique, {}, {} \n'.format(path, x.shape[0]))

    if (mode == 1 and x.shape[0] > 100000) or (mode == 2 and x.shape[0] > 50000):
            temp = x.shape[0] // 10
            start = sliceno * temp
            end = start + temp - 1
            x = x[start:end,:]
            with open(folder_name + 'instances_count.csv','a') as f:
                f.write('Start, {}, End, {} \n'.format(start, end))
    elif mode == 0:
        if x.shape[0] > 15000000:
            temp = x.shape[0] // 400
            start = sliceno * temp
            end = start + temp - 1
            x = x[start:end,:]
            with open(folder_name + 'instances_count.csv','a') as f:
                f.write('Start, {}, End, {} \n'.format(start, end))
        elif x.shape[0] > 10000000:
            temp = x.shape[0] // 200
            start = sliceno * temp
            end = start + temp - 1
            x = x[start:end,:]
            with open(folder_name + 'instances_count.csv','a') as f:
                f.write('Start, {}, End, {} \n'.format(start, end))
        elif x.shape[0] > 100000:
            temp = x.shape[0] // 10
            start = sliceno * temp
            end = start + temp - 1
            x = x[start:end,:]
            with open(folder_name + 'instances_count.csv','a') as f:
                f.write('Start, {}, End, {} \n'.format(start, end))


    y = np.full(x.shape[0], label)

    with open(folder_name + 'instances_count.csv','a') as f:
        f.write('slice, {}, {} \n'.format(path, x.shape[0]))

    return x, y

In [ ]:
#training

def classify_sub(classifier, x_train, y_train, x_test, y_test, cm_file_name, summary_file_name, classifier_name, verbose = True):
  classifier.fit(x_train, y_train)
  pred = classifier.predict(x_test)

  cm = pd.crosstab(y_test, pred)
  cm.to_csv(cm_file_name)

  pd.DataFrame(classification_report(y_test, pred, output_dict = True)).transpose().to_csv(summary_file_name)

  if verbose:
      print(classifier_name + ' Done.\n')

  del classifier
  del pred
  del cm

def classify(random_state, x_train, y_train, x_test, y_test, folder_name, prefix = "", verbose = True):
    confusion_matrix_folder = os.path.join(folder_name, 'Confusion_Matrix/')
    summary_folder =  os.path.join(folder_name, 'Summary/')

    if os.path.isdir(confusion_matrix_folder) == False:
            os.mkdir(confusion_matrix_folder)
    if os.path.isdir(summary_folder) == False:
            os.mkdir(summary_folder)

    # MLP
    def classify_mlp(random_state, x_train, y_train, x_test, y_test, folder_name, prefix="", verbose=True):
        confusion_matrix_folder = os.path.join(folder_name, 'Confusion_Matrix/')
        summary_folder = os.path.join(folder_name, 'Summary/')

        if os.path.isdir(confusion_matrix_folder) == False:
                os.mkdir(confusion_matrix_folder)
        if os.path.isdir(summary_folder) == False:
                os.mkdir(summary_folder)



    # # 1- Linear
    linear_classifier = LogisticRegression(random_state = random_state)
    classify_sub(linear_classifier,
                 x_train, y_train,
                 x_test, y_test,
                 confusion_matrix_folder + prefix + '_cm_linear.csv',
                 summary_folder + prefix + '_summary_linear.csv',
                 'Linear',
                 verbose)

    # 2- KNN
    knn_classifier = KNeighborsClassifier()
    classify_sub(knn_classifier,
                 x_train, y_train,
                 x_test, y_test,
                 confusion_matrix_folder + prefix + '_cm_knn.csv',
                 summary_folder + prefix + '_summary_knn.csv',
                 'KNN',
                 verbose)

    #3- RBF SVM
    kernel_svm_classifier = SVC(kernel = 'rbf', random_state = random_state, gamma='scale')
    classify_sub(kernel_svm_classifier,
                 x_train, y_train,
                 x_test, y_test,
                 confusion_matrix_folder + prefix + '_cm_kernel_svm.csv',
                 summary_folder + prefix + '_summary_kernel_svm.csv',
                 'SVM',
                 verbose)

    #4- Naive Bayes
    naive_classifier = GaussianNB()
    classify_sub(naive_classifier,
                 x_train, y_train,
                 x_test, y_test,
                 confusion_matrix_folder + prefix + '_cm_naive.csv',
                 summary_folder + prefix + '_summary_naive.csv',
                 'Naive',
                 verbose)

    #5- Decision Tree
    decision_tree_classifier = DecisionTreeClassifier(criterion = 'entropy', random_state = random_state)
    classify_sub(decision_tree_classifier,
                 x_train, y_train,
                 x_test, y_test,
                 confusion_matrix_folder + prefix + '_cm_decision_tree.csv',
                 summary_folder + prefix + '_summary_decision_tree.csv',
                 'Decision Tree',
                 verbose)

    #6- Random Forest
    random_forest_classifier = RandomForestClassifier(n_estimators = 10, criterion = 'entropy', random_state = random_state)
    classify_sub(random_forest_classifier,
                 x_train, y_train,
                 x_test, y_test,
                 confusion_matrix_folder + prefix + '_cm_random_forest.csv',
                 summary_folder + prefix + '_summary_random_forest.csv',
                 'Random Forest',
                 verbose)

    # 7- Linear SVM
    svm_classifier = LinearSVC(random_state = random_state)
    classify_sub(svm_classifier,
                 x_train, y_train,
                 x_test, y_test,
                 confusion_matrix_folder + prefix + '_cm_svm.csv',
                 summary_folder + prefix + '_summary_svm.csv',
                 'SVM',
                 verbose)

      # 8- Neural Network
    mlp_classifier = MLPClassifier(hidden_layer_sizes=([100, 100, 100]), alpha=0.001)
    classify_sub(mlp_classifier,
                 x_train, y_train,
                 x_test, y_test,
                 confusion_matrix_folder + prefix + '_cm_mlp.csv',
                 summary_folder + prefix + '_summary_mlp.csv',
                 'MLP',
                 verbose)

    xgb_classifier = XGBClassifier(learning_rate=0.4, max_depth=7)
    classify_sub(xgb_classifier,
                 x_train, y_train,
                 x_test, y_test,
                 confusion_matrix_folder + prefix + '_cm_xbg.csv',
                 summary_folder + prefix + '_summary_xbg.csv',
                 'XBG',
                 verbose)


In [ ]:
if __name__ == "__main__":
    parser = argparse.ArgumentParser()
    parser.add_argument('--mode', type = int, default = 2)
    parser.add_argument('--output', default='Classification_Bi')
    parser.add_argument('--verbose', type = str2bool, default = True)
    parser.add_argument("-f", "--file", required=False)

    args = parser.parse_args()

    for slice_number in range(11):
        prefix = ''
        if args.mode == 1:
            prefix = 'uniflow_'
        elif args.mode == 2:
            prefix = 'biflow_'

        if args.verbose:
            print('Starting Slice #: {}'.format(slice_number))
            print('Start Classification')

        random_state = 0
        folder_name = '{}_{}/'.format(args.output, slice_number)

        if os.path.isdir(folder_name) == False:
            os.mkdir(folder_name)

        x, y = load_file('/content/drive/MyDrive/DS-MQTT/packet_features/mqtt_bruteforce.csv',
                         args.mode,
                         0, 0,
                         folder_name,
                         slice_number,
                         args.verbose)


        x_temp, y_temp = load_file('/content/drive/MyDrive/DS-MQTT/biflow_features/biflow_scan_A.csv',
                                   args.mode,
                                   1, 1,
                                   folder_name,
                                   slice_number,
                                   args.verbose)

        x = np.concatenate((x, x_temp), axis = 0)
        y = np.append(y, y_temp)
        del x_temp, y_temp

        x_temp, y_temp = load_file('/content/drive/MyDrive/DS-MQTT/biflow_features/biflow_scan_sU.csv',
                                   args.mode,
                                   1, 2,
                                   folder_name,
                                   slice_number,
                                   args.verbose)

        x = np.concatenate((x, x_temp), axis = 0)
        y = np.append(y, y_temp)
        del x_temp, y_temp

        x_temp, y_temp = load_file('/content/drive/MyDrive/DS-MQTT/biflow_features/biflow_sparta.csv',
                                   args.mode,
                                   1, 3,
                                   folder_name,
                                   slice_number,
                                   args.verbose)

        x = np.concatenate((x, x_temp), axis = 0)
        y = np.append(y, y_temp)
        del x_temp, y_temp

        x_temp, y_temp = load_file('/content/drive/MyDrive/DS-MQTT/biflow_features/biflow_mqtt_bruteforce.csv',
                                   args.mode,
                                   1, 4,
                                   folder_name,
                                   slice_number,
                                   args.verbose)

        x = np.concatenate((x, x_temp), axis = 0)
        y = np.append(y, y_temp)
        del x_temp, y_temp

        x_train, x_test, y_train, y_test = train_test_split(x, y,
                                                            test_size = 0.25,
                                                            random_state = 42)

        classify(random_state, x_train, y_train, x_test, y_test,
                 folder_name, "slice_{}_no_cross_validation".format(slice_number), args.verbose)

        kfold = StratifiedKFold(n_splits = 5, shuffle = True, random_state = 0)

        counter = 0
        for train, test in kfold.split(x, y):
            classify(random_state, x[train], y[train], x[test], y[test],
                     folder_name, "slice_{}_k_{}".format(slice_number, counter), args.verbose)
            counter += 1

        del x
        del y
        del x_train
        del x_test
        del y_train
        del y_test

Starting Slice #: 0
Start Classification


<ipython-input-10-983d24d6d27a>:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x_temp.drop('is_attack', axis = 1, inplace = True)


TypeError: ignored

In [ ]:
print("Prefix:", prefix)
file_path = prefix + 'normal.csv'
print("Full File Path:", file_path)


Prefix: biflow_
Full File Path: biflow_normal.csv


In [ ]:
import os

if os.path.exists('biflow_scan_A.csv'):
    print('File exists')
else:
    print('File does not exist')

File does not exist


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!cp -r /content/Classification_Bi_9 /content/drive/MyDrive/